# When Explanations Lie: Why Modified BP Attribution fails

This notebook produces the cosine similaries of the relevance vectors.

In [ ]:
# uncomment to install install packages
# !pip install tensorflow-gpu=1.13.1
# !pip install innvestigate seaborn tqdm

In [ ]:
%env CUDA_VISIBLE_DEVICES=1

In [ ]:
import tensorflow
import tensorflow as tf
import warnings

import innvestigate
import matplotlib.pyplot as plt

import numpy as np
import PIL 
import copy
import contextlib

import imp
import numpy as np
import os

from skimage.measure import compare_ssim 
import pickle
from collections import OrderedDict
from IPython.display import IFrame, display

import keras
import keras.backend
import keras.models


import innvestigate
import innvestigate.applications.imagenet
import innvestigate.utils as iutils
import innvestigate.utils as iutils
import innvestigate.utils.visualizations as ivis
from innvestigate.analyzer.relevance_based.relevance_analyzer import LRP
from innvestigate.analyzer.base import AnalyzerNetworkBase, ReverseAnalyzerBase
from innvestigate.analyzer.deeptaylor import DeepTaylor

import time
import tqdm

import seaborn as sns

import itertools
import matplotlib as mpl

from tensorflow.python.client import device_lib

In [ ]:
def _prepare_model(self, model):
    return super(DeepTaylor, self)._prepare_model(model)

# otherwise DTD does not work on negative outputs
DeepTaylor._prepare_model = _prepare_model

In [ ]:
# device_lib.list_local_devices()

In [ ]:
# path to imagenet validation
imagenet_val_dir = "/mnt/ssd/data/imagenet/imagenet-raw/validation"
# path to examplary image
ex_image_path = "n01534433/ILSVRC2012_val_00015410.JPEG"
# number of images to run the evaluation
n_selected_imgs = 200

model_names = ['resnet50', 'vgg16']

In [ ]:
# helper functions 

def load_image(path, size):
    ret = PIL.Image.open(path)
    ret = ret.resize((size, size))
    ret = np.asarray(ret, dtype=np.uint8).astype(np.float32)
    if ret.ndim == 2:
        ret.resize((size, size, 1))
        ret = np.repeat(ret, 3, axis=-1)
    return ret

def preprocess(X, net):
    X = X.copy()
    X = net["preprocess_f"](X)
    return X

def image(X):
    X = X.copy()
    return ivis.project(X, absmax=255.0)

def to_heatmap(saliency):
    return ivis.heatmap(
        np.abs(
            ivis.clip_quantile(saliency.copy(), 0.5))
        )

def copy_weights(model_to, model_from, idxs):
    idxs = set(idxs)
    all_weights = []
    for i in range(len(model_to.layers)):
        if i in idxs:
            all_weights.extend(model_from.layers[i].get_weights())
        else:
            all_weights.extend(model_to.layers[i].get_weights())
    model_to.set_weights(all_weights)
        
def get_random_target(target_exclude):
    return np.random.choice([i for i in range(1000) if i != target_exclude])

def normalize(x, percentile=99):
    """
    all heatmap are normalized
    """
    vmin = np.percentile(x, percentile)
    vmax = np.percentile(x, 100 - percentile)
    return np.clip((x - vmin) / (vmax - vmin), 0, 1)

def ssim_flipped(x, y, win_size=5, **kwargs):
    norm_x = normalize(x)
    norm_y = normalize(y)
    norm_y_flip = normalize(-y)
    
    ssim = compare_ssim(norm_x, norm_y, win_size=win_size, **kwargs)
    ssim_flip = compare_ssim(norm_x, norm_y_flip, win_size=win_size, **kwargs)
    return max(ssim, ssim_flip)


def l2_flipped(x, y):
    norm_x = normalize(x)
    norm_y = normalize(y)
    norm_y_flip = normalize(-y)
    l2 = np.mean(np.sqrt((norm_x - norm_y)**2))
    l2_flipped = np.mean(np.sqrt((norm_x - norm_y_flip)**2))
    return max(l2, l2_flipped)


def norm_image(x):
    mi = x.min()
    ma = x.max()
    return (x - mi) / (ma - mi)

def load_image_paths(validation_dir):
    val_filepaths = ! find {validation_dir} -name '*.JPEG' 
    val_filepaths = sorted(val_filepaths)
    val_targets = ! ls {validation_dir}
    val_targets = sorted(val_targets)
    
    val_path_with_target = []
    for path in val_filepaths:
        synnet = path.split('/')[-2]
        target = val_targets.index(synnet)
        val_path_with_target.append((path, target))
    return val_path_with_target




In [ ]:
def load_model(model='vgg16', load_weights=True, load_patterns="relu"):
    load_func = getattr(innvestigate.applications.imagenet, model)
    net = load_func(load_weights=load_weights, load_patterns=load_patterns)
    model = keras.models.Model(inputs=net["in"], outputs=net["sm_out"])
    model_wo_softmax = iutils.keras.graph.model_wo_softmax(model)
    
    channels_first = keras.backend.image_data_format() == "channels_first"
    color_conversion = "BGRtoRGB" if net["color_coding"] == "BGR" else None
    return model_wo_softmax, net, color_conversion

def get_output_shapes(model, input_shape=(None, 224, 224, 3)):
    model_output_shapes = OrderedDict()
    for i, layer in enumerate(model.layers):
        model_output_shapes[i] = layer.get_output_shape_at(0)
    return model_output_shapes

In [ ]:
def get_nice_resnet_layers(model):
    def get_conv_name(layer):
        block, branch = layer.name.split('_')
        block_spec = block.lstrip('res')
        block_idx = block_spec[0]
        subblock_idx = ' abcdefg'.index(block_spec[1])
        layer_idx = branch[-1]
        if layer_idx == '1':
            layer_idx = 's'
            nice_name = "conv{}_skip".format(block_idx)
        else:
            nice_name = "conv{}_{}{}".format(block_idx, subblock_idx, layer_idx) 
        return nice_name
        
    nice_layer_names_resnet = OrderedDict()
    for i in range(len(model.layers)):
        layer = model.get_layer(index=i)
        if type(layer) == keras.layers.Conv2D:
            if layer.name == 'conv1':
                nice_layer_names_resnet[i] = 'conv1'
                continue

            nice_layer_names_resnet[i] = get_conv_name(layer)
            #print(block_spec, block_idx, subblock_idx, layer.name, nice_name)
        elif type(layer) == keras.layers.MaxPool2D:
            nice_layer_names_resnet[i] = 'maxpool'
        elif type(layer) == keras.layers.GlobalAveragePooling2D:
            nice_layer_names_resnet[i] = 'avgpool'
        elif type(layer) == keras.layers.Add:
            prev_layer = model.get_layer(index=i-1)
            
            block, branch = prev_layer.name.split('_')
            block_spec = block.lstrip('bn')
            block_idx = block_spec[0]
            subblock_idx = ' abcdefg'.index(block_spec[1])
            nice_name = 'block{}_{}'.format(block_idx, subblock_idx)
            # print(prev_layer.name, nice_name, block_idx, subblock_idx, 
            #       model.get_layer(index=i+1).name)
            print(nice_name)
            # name the following activation layers
            nice_layer_names_resnet[i+1] = nice_name
        elif type(layer) == keras.layers.Dense:
            nice_layer_names_resnet[i] = 'dense'
    return nice_layer_names_resnet


def get_layer_idx(model_name, layer_name):
    for key, l in nice_layer_names[model_name].items():
        if l == layer_name:
            return key
        
    raise ValueError("no layer: " + layer_name)

In [ ]:
val_paths = load_image_paths(imagenet_val_dir)

ex_image_full_path, ex_target = [
    (path, target) for (path, target) in val_paths 
    if path.endswith(ex_image_path)][0]
ex_image_idx = val_paths.index((ex_image_full_path, ex_target))

np.random.seed(0)

selected_img_idxs = [ex_image_idx] + np.random.choice([idx for idx in range(len(val_paths))
                                                       if idx != ex_image_idx], n_selected_imgs - 1).tolist()

In [ ]:
# Load the model definition.

model, innv_net, color_conversion = load_model('resnet50')

In [ ]:
nice_layer_names = {
    'vgg16': OrderedDict([
        (0, 'input'),
        (1, 'conv1_1'),
        (2, 'conv1_2'),
        (3, 'pool1'),
        (4, 'conv2_1'),
        (5, 'conv2_2'),
        (6, 'pool2'),
        (7, 'conv3_1'),
        (8, 'conv3_2'),
        (9, 'conv3_3'),
        (10, 'pool3'),
        (11, 'conv4_1'),
        (12, 'conv4_2'),
        (13, 'conv4_3'),
        (14, 'pool4'),
        (15, 'conv5_1'),
        (16, 'conv5_2'),
        (17, 'conv5_3'),
        (18, 'pool5'),
        (19, 'flatten'),
        (20, 'fc1'),
        (21, 'fc2'),
        (22, 'fc3') 
    ]),
    'resnet50': get_nice_resnet_layers(model)
}

In [ ]:
n_layers = {'vgg16': 22, 'resnet50': 177}

replacement_layers = {
    'vgg16':  ['fc3', 'fc1', 'conv4_3', 'conv3_3', 'conv2_2'],
    'resnet50': ['dense', 'block5_1', 'block4_2', 'block3_4', 'block3_2', 'block2_2'],
}

In [ ]:
ex_image = preprocess(load_image(os.path.join(imagenet_val_dir, ex_image_path), 224),
                      innv_net)[np.newaxis]
ex_target = [target for (path, target) in val_paths 
             if path.endswith(ex_image_path)][0]

val_images = [(
    preprocess(
        load_image(val_paths[idx][0], 224),
        innv_net
    )[np.newaxis],
    val_paths[idx][1]) for idx in selected_img_idxs]

print("Loaded {} images".format(len(val_images)))


output_shapes = get_output_shapes(model)

print_output_shapes = False 
if print_output_shapes: 
    print("{:3}{:20}{:20}{}".format("l", "layer", "input_at_0", "output_shape"))
    for i in range(len(model.layers)):
        layer = model.get_layer(index=i)
        print("{:3}: {:20}  {:20}  {}".format(
            i, layer.name, str(layer.get_input_shape_at(0)), str(output_shapes[i])))
        #print("{:3}: {:20}  {:20}  {}".format(i, type(layer).__name__, layer.name, output_shapes[i]))

In [ ]:
model_cascading, _, _ = load_model('resnet50')
model_random, _, _ = load_model('resnet50', load_weights=False)
model_cascading.set_weights(model.get_weights())
out = model.predict(ex_image)
out_cascading = model_cascading.predict(ex_image)
print("mean-l1 distance of the outputs of the trained model and when weights are from trained model [should be 0]:", np.abs(out_cascading - out).mean())

n_layers = len(model_random.layers)
copy_weights(model_cascading, model_random, range(n_layers - 3, n_layers))

out = model.predict(ex_image)
out_cascading = model_cascading.predict(ex_image)
print( "mean-l1 distance of the outputs of the trained model when the last 2 layers are random [should not be 0]:", np.abs(out_cascading - out).mean())

In [ ]:
def postprocess(name, x):
    if name == 'abs.max':
        return np.abs(x).max(-1)
    elif name == 'abs.sum':
        return np.abs(x).sum(-1)
    elif name == 'sum':
        return x.sum(-1)
    else:
        pass
    

input_range = (ex_image.min(), ex_image.max())
noise_scale = 0.15 * (ex_image.max() - ex_image.min())

# label, innv_name, postprocess, exclude, kwargs


attr_names = [n for (n, _, _, _, _) in analysers]

hmap_postprocesisng = {
    n: lambda x: postprocessing(post_name, x) for n, _, post_name, _, _ in analysers
}

In [ ]:
mpl_linestyle = ['solid', 'dashed', 'dotted']
#    "o", #circle
#    "v", #triangle_down
#    "^", #triangle_up
#    "X", #star
#    "s", #square
#    "<", #triangle_left
#    "P", #plus (filled)
#    "D", #diamond
#    "$O$", 
#    ">",
#    "$V$", #hexagon2
#    "$P$", 
#    "$S$"

color_palette = sns.color_palette('colorblind', n_colors=5)
hatches = [None, '//']
styles = list(itertools.product(mpl_linestyle, zip(mpl_markers, itertools.cycle(color_palette))))

mpl_styles = OrderedDict([
    ('GuidedBP',                   {'marker': 'X',   'color': colors[0]}),
    ('Deconv',                     {'marker': '$D$', 'color': colors[1]}),
    ('LRP-z',                      {'marker': 'D',   'color': colors[2]}),
    ('DTD',                        {'marker': '$T$', 'color': colors[3]}),
    ('PatternAttr.',               {'marker': '$P$', 'color': colors[4]}),
    ('LRP-$\\alpha=1, \\beta=0$',  {'marker': '<',   'color': colors[0]}),
    ('LRP-$\\alpha=2, \\beta=1$',  {'marker': '>',   'color': colors[1]}),
    ('LRP-$\\alpha=5, \\beta=4$',  {'marker': '^',   'color': colors[2]}),
    ('LRP-cmp-$\\alpha=1$',        {'marker': 's',   'color': colors[3]}),
    ('LRP-cmp-$\\alpha=2$',        {'marker': 'P',   'color': colors[4]}),
    ('SmoothGrad',                 {'marker': 'o',   'color': colors[5]}),
    ('Gradient',                   {'marker': 'v',   'color': 'black'}),
])



print(len(styles), len(attr_names))
assert len(styles) >= len(attr_names)

linestyles = OrderedDict(
    [(name, {'linestyle': 'solid', 'marker': marker_dict[name], 'color': c}) 
     for name, (l, (m, c)) in zip(attr_names, styles)]
)
if 'Gradient' in linestyles:
    linestyles['Gradient']['color'] = 'black'
    
for i, (name, style) in enumerate(mpl_styles.items()):
    plt.plot(np.arange(10), [20-i] * 10, 
             #markersize=5,
             label=name + " m=" + linestyle['marker'], **linestyle)
    
plt.legend(bbox_to_anchor=(1, 1))

In [ ]:
def create_replacement_class(analyser_cls):
    assert issubclass(analyser_cls, ReverseAnalyzerBase)
    class ReplaceBackward(analyser_cls):
        def __init__(self, model, replace_layer, replace_shape, *args, **kwargs):
            kwargs['reverse_keep_tensors'] = True
            super().__init__(model, *args, **kwargs)
            self._replace_shape = replace_shape
            self._replace_layer = replace_layer
            self._replace_tensor = None

        def _prepare_model(self, model):
            model, analysis_inputs, stop_analysis_at_tensor = super()._prepare_model(model)
            self._replace_tensor = keras.layers.Input(name='replace_backward', 
                                                      batch_shape=self._replace_shape)   
            self._replace_inputs = analysis_inputs + [self._replace_tensor]
            return model, analysis_inputs + [self._replace_tensor], stop_analysis_at_tensor

        def _create_analysis(self, *args, **kwargs):
            def check_layer(layer):
                return layer == self._replace_layer

            def replace_backward(Xs, Ys, Rs, reverse_state):               
                return [keras.layers.Lambda(lambda x: 2*x / 2)(self._replace_tensor)]

            self._add_conditional_reverse_mapping(check_layer, replace_backward)
            outputs, intermediate = super()._create_analysis(*args, **kwargs)
            self._create_cos_model(intermediate)
            return outputs, intermediate
        
        def get_cosine(self, X, replacement, intermediate_values):
            sess = keras.backend.get_session()
            feed_dict = OrderedDict([
                (t, v) for t, v in zip(self._intermediate_references, intermediate_values)
            ])
            feed_dict[self._replace_tensor] = replacement
            
            tf_X = repl_analyser._analyzer_model.inputs[0]
            feed_dict[tf_X] = X
            return sess.run(self._cosine_similarities, feed_dict=feed_dict)
            
        def get_cosine_grad(self, X, replacement, intermediate_values):
            sess = keras.backend.get_session()
            feed_dict = OrderedDict([
                (t, v) for t, v in zip(self._intermediate_references, intermediate_values)
            ])
            feed_dict[self._replace_tensor] = replacement
            
            tf_X = repl_analyser._analyzer_model.inputs[0]
            feed_dict[tf_X] = X
            return sess.run(self._cosine_grads, feed_dict=feed_dict)
            
            
        def _create_cos_model(self, intermediate_tensors):
            self._intermediate_tensors = intermediate_tensors[1:]
            self._intermediate_references = [
                keras.layers.Input(name='intermediate_{}'.format(i), batch_shape=tens.shape)
                for i, tens in enumerate(self._intermediate_tensors)
            ]
            self._replace_inputs 
            
            self._cosine_similarities = [] 
            
            for v, r in zip(self._intermediate_tensors, self._intermediate_references):
                
                r_norm = tf.nn.l2_normalize(r, axis=-1)
                v_norm = tf.nn.l2_normalize(v, axis=-1)
                cos = 1 - tf.losses.cosine_distance(r_norm, v_norm, axis=-1, reduction='none')
                self._cosine_similarities.append(cos)
                
            cos_sim = self._cosine_similarities[-1]
            self._cosine_grads = [g for g in tf.gradients(tf.reduce_mean(cos_sim), self._intermediate_tensors) if g is not None]
            
    return ReplaceBackward 


def get_replacement_analyser(model, analyser_cls, replacement_layer_idx, model_output_shapes=None, **kwargs):
    if type(analyser_cls) == str:
        analyser_cls = innvestigate.analyzer.analyzers[analyser_cls]
    replacement_cls = create_replacement_class(analyser_cls)
    
    if model_output_shapes is None:
        model_output_shapes = get_output_shapes(model)
    
    replacement_shape = model_output_shapes[replacement_layer_idx - 1]
    replace_layer = model.layers[replacement_layer_idx]
    return replacement_cls(model, replace_layer, replacement_shape, **kwargs), replacement_shape

In [ ]:
%pdb off

In [ ]:
nice_layer_names

In [ ]:
n_layers = {'vgg16': 23, 'resnet50': 177}

In [ ]:
# replacement_layer_indices = [22]
n_sampled_v = 5

cos_sim_median = OrderedDict()
selected_percentiles = [0, 1, 5, 10, 20, 50, 100]
cos_sim_percentiles = OrderedDict()
for label, innv_name, _, excludes, kwargs in tqdm.tqdm_notebook(analysers):
    if 'exclude_cos_sim' in excludes:
        continue
        
    for model_name in ['resnet50', 'vgg16']:
        if 'exclude_' + model_name in excludes:
            continue
        keras.backend.clear_session()
        model_wo_softmax, innv_net, _ = load_model(model_name, load_weights=True)
        if innv_name == "pattern.attribution":
            kwargs['patterns'] = innv_net['patterns']

        for replacement_layer in replacement_layers[model_name][:]:
            replacement_layer_idx = get_layer_idx(model_name, replacement_layer)
            repl_analyser, repl_shape = get_replacement_analyser(
                model_wo_softmax, innv_name,  
                replacement_layer_idx=replacement_layer_idx,
                **kwargs)
            repl_analyser.create_analyzer_model()
            cos_per_img = OrderedDict()
            for img_idx, (img, _) in tqdm.tqdm_notebook(
                zip(selected_img_idxs, val_images), 
                desc="[{}.{}] {}".format(model_name, replacement_layer, label)):
                channels = repl_shape[-1]
                if label == "$\\alpha=100, \\beta=99$-LRP":
                    # a=100,b=99 sufferes numerical instabilities with std = 1
                    std = 1 / np.sqrt(channels)
                else:
                    std = 1

                replacement = std*np.random.normal(size=(1, ) + repl_shape[1:]) 
                hmap = repl_analyser.analyze([img, replacement])
                intermediate_values = repl_analyser._reversed_tensors

                replacement = std * np.random.normal(size=(n_sampled_v,) + repl_shape[1:]) 
                outs = repl_analyser.get_cosine(np.tile(img, (n_sampled_v, 1, 1, 1)), replacement, [v for (_, v) in intermediate_values[1:][::-1]])
                for layer_idx, o in enumerate(outs):
                    cos_per_img[model_name, layer_idx, img_idx] = np.abs(o)

            median_for_label = []
            percentile_for_label = OrderedDict([(p, []) for p in selected_percentiles])
            for layer_idx in range(n_layers[model_name]):
                cos_per_layer = np.concatenate([cos_per_img[model_name, layer_idx, img_idx]  for img_idx in selected_img_idxs])
                perc_values = np.percentile(cos_per_layer.flatten(),  selected_percentiles)
                for p, val in zip(selected_percentiles, perc_values):
                    percentile_for_label[p].append(val)

            for p, values in percentile_for_label.items():
                cos_sim_percentiles[label, model_name, replacement_layer_idx, p] = np.array(values)

In [ ]:
save_results = True
if save_results:
    with open('cache/cos_sim_with_resnet.pickle', 'wb') as f:
        pickle.dump((cos_sim_percentiles), f)

In [ ]:
def cosine_similarity(U, V):
    v_norm = V / np.linalg.norm(V, axis=0, keepdims=True)
    u_norm = U / np.linalg.norm(U, axis=0, keepdims=True)
    return v_norm.T @ u_norm

def get_sample_cos_sim_per_layer(output_shapes):
    values = []
    for layer_idx, shp in output_shapes.items():
        ch = shp[-1]
        n_samples = 1000
        u = np.random.normal(size=(ch, n_samples))
        v = np.random.normal(size=(ch, n_samples))
        cos = cosine_similarity(u, u)
        mask = np.tri(cos.shape[0])
        values.append(np.median(np.abs(cos[mask == 1])))
    return np.array(values)
        

In [ ]:
[n for (n, _, _, _, _) in analysers]

In [ ]:
legend = OrderedDict()
for model_name in model_names[::-1]:
    
    keras.backend.clear_session()
    model, _, _ = load_model(model_name)
    output_shapes = get_output_shapes(model)

    sample_cos_per_layer = get_sample_cos_sim_per_layer(output_shapes)
    for replacement_layer in replacement_layers[model_name]:
        repl_idx = get_layer_idx(model_name, replacement_layer)
        start_layer = n_layers[model_name] - repl_idx
        
        layer_names = [name for idx, name in nice_layer_names[model_name].items()
                       if idx <= repl_idx][::-1]
        layer_idxs = np.array([idx for idx, name in nice_layer_names[model_name].items()
                       if idx < repl_idx][::-1])
        # print(layer_idxs)
        displayed_layers = n_layers[model_name] - layer_idxs
        plt.figure(figsize=(max(3, len(displayed_layers) / 4), 3.5))
        for i, (label, _, _, _, _) in enumerate(analysers):
            idx = (label, model_name, repl_idx, 50)
            if idx not in cos_sim_percentiles:
                warnings.warn("not found: " + str(idx))
                continue
            #print(len(cos_sim_percentiles[idx]), repl_idx, displayed_layers)
            cos_sim_per_label = cos_sim_percentiles[idx][displayed_layers]
            plt.plot(0.5 + np.arange(len(cos_sim_per_label)), cos_sim_per_label, label=label, **linestyles[label])
            if label not in legend:
                legend[label] = linestyles[label]
            
        layer_names = [name for idx, name in nice_layer_names[model_name].items()
                       if idx <= repl_idx][::-1]
        
        
        # Random Cos Similarity
        # Cos Similarity Base.
        label='Cos Similarity BL'
        style = {'color': (0.25, 0.25, 0.25)}
        plt.plot(0.5 + np.arange(len(displayed_layers)), sample_cos_per_layer[::-1][displayed_layers], 
                 # label='Cos. Sim. Baseline', 
                 label=label,
                 **style)
        if label not in legend:
            legend[label] = style
        
        #plt.legend(bbox_to_anchor=(1, 1))
        plt.ylabel('cosine similarity')
        plt.xticks(np.arange(len(layer_names)), layer_names, rotation=90)
        plt.ylim(-0.05, 1.05)
        plt.grid('on', alpha=0.35) #, axis="y")
        plt.savefig("./figures/consine_similarity_{}_layer_{}.pdf".format(model_name, repl_idx),  
                    bbox_inches='tight', pad_inches=0)
        plt.show()
        plt.close()

In [ ]:

for replacement_layer_idx in replacement_layer_indices:
    display(IFrame("./figures/consine_similarity_vgg16_layer_{}.pdf".format(replacement_layer_idx), 800, 600))
    

In [ ]:
plt.figure(figsize=(2, 1))
for label, style in legend.items():
    plt.plot([], [], label=label, **style)

plt.axis('off')
plt.legend(loc='center')
#plt.tight_layout()
plt.savefig('figures/cos_sim_legend.pdf', 
            bbox_inches='tight', pad_inches=0)

In [ ]:
display(IFrame("./figures/cos_sim_legend.pdf", 800, 600))

In [ ]:
raise Exception('stop')

In [ ]:
## PatternAttribution

In [ ]:


layers_w_kernel = ([l for l in model.layers if type(l) == keras.layers.Conv2D]
                    + [l for l in model.layers if type(l) == keras.layers.Dense])
for i in range(len(patterns)):
    w = layers_w_kernel[i].get_weights()[0]
    pw = patterns[i] * w
    print(i, (patterns[i] < 0).sum() / patterns[i].size, (w < 0).sum() / patterns[i].size, (pw > 0).sum() / pw.size)

### Why does GBP and Deconv converge SOOOO fast

In [ ]:

gdb = ('GuidedBP', 
       "guided_backprop", 
         lambda x: np.abs(x).sum(-1), 
         {})
deconv = ('Deconv', 
         "deconvnet", 
         lambda x: np.abs(x).sum(-1), 
         {})

replacement_layer_idx = 22
n_sampled_v = 10
cos_sim = OrderedDict()
for label, innv_name, postpro, kwargs in tqdm.tqdm_notebook([gdb, deconv]):
    repl_analyser, repl_shape = get_replacement_analyser(model_wo_softmax, innv_name, 
                                                         replacement_layer_idx=replacement_layer_idx,
                                                         **kwargs)
    repl_analyser.create_analyzer_model()
    
    
    analyser = innvestigate.create_analyzer(innv_name, model_wo_softmax, 
                                             reverse_keep_tensors= True, 
                                             **kwargs)
    analyser.create_analyzer_model()
    for img_idx, (img, _) in tqdm.tqdm_notebook(zip(selected_img_idxs, val_images), desc=label):
        channels = repl_shape[-1]
        if label == "$\\alpha=100, \\beta=99$-LRP":
            # a=100,b=99 sufferes numerical instabilities with std = 1
            std = 1 / np.sqrt(channels)
        else:
            std = 30

        replacement = std*np.random.normal(size=(5, ) + repl_shape[1:]) 
        img_tiled = np.tile(img, (5, 1, 1, 1))
        hmap = repl_analyser.analyze([img_tiled, replacement])
        
        repl_hidden = [h for (_, h) in repl_analyser._reversed_tensors[1:]]
        _ = analyser.analyze(img)
        plain_hidden = [h for (_, h) in analyser._reversed_tensors[1:]]

        #replacement = std * np.random.normal(size=(n_sampled_v,) + repl_shape[1:]) 
        #outs = repl_analyser.get_cosine(np.tile(img, (10, 1, 1, 1)), replacement, [v for (_, v) in intermediate_values[1:][::-1]])
        #cos_sim[label, replacement_layer_idx, img_idx] = [np.abs(o) for o in outs]
        break
    break

In [ ]:
def cosine_similarity(U, V):
    v_norm =  V / np.linalg.norm(V, axis=0, keepdims=True)
    u_norm = U / np.linalg.norm(U, axis=0, keepdims=True)
    return (v_norm.T @ u_norm)

def relu(x):
    return np.maximum(0, x)

In [ ]:
def kernel_as_matrix(w):
    kh, kw, cin, cout = w.shape
    return w.reshape(kh*kw*cin, cout) 

for layer_name, layer in zip(nice_layer_names, model_wo_softmax.layers):
    try:
        w, b = layer.get_weights()
    except ValueError:
        continue
    if len(w.shape) == 4:
        w = kernel_as_matrix(w)
        
    #s = np.linalg.eigvals(w.T)
    if False:
        u, s, v = np.linalg.svd(w.T)
        plt.title(layer_name)
        plt.plot(s[np.argsort(s)])
        plt.show()
        print(layer_name, s.min(), s.max())

    cin, cout = w.shape
    print(w.shape)
    plt.title(layer_name)
    plt.hist(((w @ relu(np.random.normal(size=(cout, 100))))).flatten(), bins=25)
    plt.show()
    
    plt.title(layer_name)
    plt.hist((relu(w @ relu(np.random.normal(size=(cout, 100))))).flatten(), bins=25)
    plt.show()


In [ ]:
w, b = model_wo_softmax.layers[-2].get_weights()
plt.hist((w.T @ plain_hidden[-2].T).flatten())

In [ ]:
plt.hist(relu(w.T @ relu(plain_hidden[-2].T)).flatten())

In [ ]:

plt.hist(w.T @ relu(plain_hidden[-2].T).flatten())


In [ ]:
plt.hist(relu(w.T @ plain_hidden[-2].T).flatten())

In [ ]:
plt.imshow(hidden_values[-3][:, :100])
plt.colorbar()

In [ ]:
plt.imshow(hidden_values[-2][:, :100])
plt.colorbar()

In [ ]:
plt.imshow((w.T @ relu(hidden_values[-2].T)).T[:, :100])
plt.colorbar()

In [ ]:
plt.imshow((w.T @ relu(hidden_values[-2].T)).T[:, :100])
plt.colorbar()

In [ ]:
plt.imshow((w.T @ (hidden_values[-2].T + b[:, None])).T[:, :100])
plt.colorbar()

In [ ]:
plt.scatter(np.arange(len(b)), b, marker='.')

In [ ]:
w, b = model.layers[-2].get_weights()
w.shape

In [ ]:

hidden_values[0].max(-1)[0].shape

In [ ]:
plt.imshow(hidden_values[0].max(-1)[4])
plt.colorbar()

In [ ]:
plt.imshow(w)
plt.colorbar()

In [ ]:
w.sum(0)

In [ ]:
w_eig = np.linalg.eigvals(w)
plt.plot(w_eig[np.argsort(w_eig)[::-1]])
print(w_eig[np.argsort(w_eig)][:10])

In [ ]:
plt.imshow(cosine_similarity(hidden_values[-3].T, hidden_values[-3].T))
plt.colorbar()

In [ ]:
[o.shape for o in outs]

In [ ]:
[np.abs(o).mean() for o in outs]

In [ ]:
hmaps = []
repr2 = []
for _ in range(128):
    replacement = np.random.normal(size=repl_shape)
    hmap = repl_analyser.analyze([ex_image, replacement])
    hmaps.append(hmap[0])
    repr2.append(repl_analyser._reversed_tensors[2][1])
    #print(hmap.min(), hmap.max())
    #plt.imshow(hmap[0].mean(-1), cmap='seismic')
    #plt.colorbar()
hmaps = np.stack(hmaps)
repr2 = np.stack(repr2)

In [ ]:
for ten in repl_analyser._reversed_tensors:
    print(ten[1].shape)

In [ ]:
repl_analyser._reverse_tensors_mapping

In [ ]:
np.corrcoef(hmaps[:, 100, 100].T)

In [ ]:
plt.imshow(repr2[:, 0, 10, 10])
plt.colorbar()

In [ ]:
plt.imshow(np.abs(np.corrcoef(repr2[:, 0, 10, 10], repr2[:, 0, 10, 10].mean(0, keepdims=True))))

In [ ]:
plt.imshow(np.abs(np.corrcoef(repr2[:, 0, 10, 10], repr2[:, 0, 10, 10].mean(0, keepdims=True))))


In [ ]:
repr2.shape

In [ ]:
plt.imshow(np.abs(np.corrcoef(repr2[:, 0, 100, 100].T)))
plt.colorbar()

In [ ]:

def normalize_neg(x):
    vmax = np.percentile(x, 99)
    return np.clip(x / vmax, -1, 1)

In [ ]:
plt.imshow(np.abs(np.corrcoef(repr2[:, 0, 90, 90].T)), vmin=0, vmax=1)
plt.colorbar()

In [ ]:
def get_median(v):
    v = tf.reshape(v, [-1])
    mid = v.get_shape()[0]//2 + 1
    return tf.nn.top_k(v, mid).values[-1]

class ReplacementGradientAnalyzer:
    def __init__(self, analyzer, with_norm=True, is_conv=True):
        if not hasattr(analyzer, '_analyzer_model'):
            analyzer.create_analyzer_model()
        self.analyzer = analyzer
        analyzer_model = self.analyzer._analyzer_model
        self.input_img, self.rk = analyzer_model.inputs
        r1 = analyzer_model.outputs[0]
        
            
        if with_norm:
            r1_perm = tf.stack([r1[:, :, :, i] for i in [1, 2, 0]], axis=-1)
            r1_reduced = tf.reduce_mean(tf.norm(r1 - r1_perm, axis=-1) / tf.norm(r1, axis=-1))
        else:
            r1_reduced = tf.reduce_mean(np.abs(r1))

        self.grad_rk, = tf.gradients([r1_reduced], [self.rk])
        
        if is_conv:
            grad_mean = tf.reduce_mean(self.grad_rk, (1, 2))
        else:
            grad_mean = self.grad_rk
            
        grad_flat = tf.reshape(grad_mean, (-1,))
        self.grad_l2 = tf.norm(grad_flat, 2)
        self.grad_l1 = tf.norm(grad_flat, 1)
        self.grad_max = tf.reduce_max(grad_flat)
        self.grad_median = get_median(grad_flat) 
        
    def r1_wrt_rk(self, input_img, relevance_k):
        sess = keras.backend.get_session() 
        return sess.run([self.grad_l2, self.grad_l1, self.grad_max, self.grad_median], {
            self.input_img: input_img, 
            self.rk: relevance_k,
        })

In [ ]:
grads_median_r1_rk = OrderedDict()
grads_max_r1_rk = OrderedDict()
grads_l2_r1_rk = OrderedDict()
grads_l1_r1_rk = OrderedDict()
replacement_shapes = OrderedDict()
for name, analyser_str, _, analyser_kwargs in analysers:
    replacements = OrderedDict()
    
    with tf.Session() as sess:
        keras.backend.set_session(sess)
        _, model_wo_softmax, _ = load_model(load_weights=True)
        for layer_name, layer_idx in tqdm.tqdm_notebook(selected_layers[::-1], desc=name):
            replacement_layer_idx = layer_idx
            if layer_idx == 1:
                replacement_layer_idx = 2
            analyzer, replacement_shape = get_replacement_analyser(
                model_wo_softmax, analyser_str,  replacement_layer_idx=layer_idx, 
                **analyser_kwargs)
            if layer_idx not in replacement_shapes:
                replacement_shapes[layer_idx] = replacement_shape
            grad_analyzer = ReplacementGradientAnalyzer(analyzer, is_conv=len(replacement_shape) == 4)
            for img_idx, (img_pp, target) in zip(selected_img_idxs, val_images):

                if (layer_idx, img_idx) not in replacements:
                    if layer_idx >= 18:
                        replacements[layer_idx, img_idx] = np.random.normal(size=replacement_shape)
                    else:
                        shp = np.array(replacement_shape)
                        shp[:-1] = 1
                        repeat = np.array(replacement_shape)
                        repeat[-1] = 1
                        replacements[layer_idx, img_idx] = np.tile(np.random.normal(size=shp), repeat)

                grad_l2, grad_l1, grad_max, grad_median = grad_analyzer.r1_wrt_rk(
                    img_pp, replacements[layer_idx, img_idx])
                grads_median_r1_rk[name, layer_idx, img_idx] = grad_median
                grads_max_r1_rk[name, layer_idx, img_idx] = grad_max
                grads_l2_r1_rk[name, layer_idx, img_idx] = grad_l2
                grads_l1_r1_rk[name, layer_idx, img_idx] = grad_l1

            del analyzer
            del grad_analyzer

In [ ]:
list(grads_max_r1_rk.keys())[0]

In [ ]:
name = '$\\alpha=1, \\beta=0$-LRP'

In [ ]:
len(grads_l2_r1_rk)

In [ ]:
layer_idx_2_name = OrderedDict([(i, n) for n, i in selected_layers])
layer_idx_2_name

In [ ]:
[(layer_idx_2_name[i], np.prod(shp)) for i, shp in replacement_shapes.items()]
n_neurons_per_layer = np.array([np.prod(shp) for i, shp in replacement_shapes.items()])
n_neurons_per_layer


In [ ]:
for name in attr_names:
    grads = np.array([[grads_l1_r1_rk[name, layer_idx, img_idx] 
                       for img_idx in selected_img_idxs] 
                      for (_, layer_idx) in selected_layers])
    #_ = plt.plot(grads * n_neurons_per_layer[::-1, None])
    _ = plt.plot(grads)
    #plt.ylim(0, 0.0001)
    plt.title(name)
    plt.xticks(ticks=np.arange(len(selected_layers)), labels=[n for (n, _) in selected_layers], rotation=90)
    plt.show()

In [ ]:
raise Exception("stop")

In [ ]:
plt.plot(list(grads_median_r1_rk.values()))

In [ ]:
plt.plot(list(grads_l2_r1_rk.values()))

In [ ]:
ReplacePatternNet = create_replacement_class(PatternNet)

In [ ]:

pattern_net = PatternNet(model_wo_softma

In [ ]:
pattern_net
%pdb off

In [ ]:
replacement_layer_idx = 10
replacement_shape = model_output_shapes[replacement_layer_idx - 1]

replace_lrp = ReplaceLRPAlpha1Beta0(model_wo_softmax, model_wo_softmax.layers[replacement_layer_idx], 
                                     model_output_shapes[replacement_layer_idx - 1])

In [ ]:

%pdb off
replace_pattern_net = ReplacePatternNet(model_wo_softmax, model_wo_softmax.layers[replacement_layer_idx], 
                                        model_output_shapes[replacement_layer_idx - 1], patterns=patterns)

replace_pattern_net.create_analyzer_model()

In [ ]:
analyzer_model = replace_pattern_net._analyzer_model
r1 = analyzer_model.outputs[0]
rl = analyzer_model.inputs[1]
r1_mean = tf.reduce_mean(r1 / tf.norm(r1, axis=-1, keepdims=True))
#r1_mean = tf.reduce_mean(np.abs(r1))
grad_rl, = tf.gradients([r1_mean], [rl])
sess = keras.backend.get_session() 
tf_input, rl = analyzer_model.inputs

grad = sess.run([grad_rl], {
    tf_input: ex_image, 
    rl: np.random.normal(size=replacement_shape)
})

## Random Weight & Logit Check

In [ ]:
model_cascading.compile(optimizer="adam", loss="categorical_crossentropy")
model_cascading_wo_softmax = iutils.keras.graph.model_wo_softmax(model_cascading)

# heatmaps are saved in those dicts
hmap_original = OrderedDict()
hmap_random_weights = OrderedDict()
hmap_random_target = OrderedDict()

for name, analyser, _, analyser_kwargs in analysers:
    cascading_heatmaps = {}
    cascading_outputs = {}
    model_cascading.set_weights(model.get_weights())
    
    last_idx = len(model.layers)
    
    analyzer_cascading = create_analyzer(analyser, model_cascading_wo_softmax, 
                                                      neuron_selection_mode="index", 
                                                      **analyser_kwargs)
    
    for img_idx, (img_pp, target) in zip(selected_img_idxs, val_images):
        random_target = get_random_target(target)
        hmap_random_target[(name, img_idx)] = (
            random_target, analyzer_cascading.analyze(img_pp, neuron_selection=random_target)[0])
    for layer_name, layer_idx in tqdm.tqdm_notebook([('original', last_idx)] + selected_layers[::-1], desc=name):
        copy_weights(model_cascading, model_random, range(layer_idx, last_idx))
        last_idx = layer_idx
        if recreate_analyser:
            analyzer_cascading = create_analyzer(
                analyser, model_cascading_wo_softmax, 
                neuron_selection_mode="index",  **analyser_kwargs)
            
        for img_idx, (img_pp, target) in zip(selected_img_idxs, val_images):
            hmap = analyzer_cascading.analyze(img_pp, neuron_selection=target)[0]
            if layer_idx == n_layers:
                hmap_original[(name, img_idx)] = hmap
            else:
                hmap_random_weights[(name, img_idx, layer_idx)] =  hmap

In [ ]:
dump_heatmaps = False

if dump_heatmaps:
    with open('heatmaps.pickle', 'wb') as f:
        pickle.dump((hmap_original, hmap_random_weights, hmap_random_target, analysers), f)

In [ ]:
# print size of dump
! ls -lh 'heatmaps.pickle'

In [ ]:
load_heatmaps = False
if load_heatmaps:
    with open('heatmaps.pickle', 'rb') as f:
        hmap_original, hmap_random_weights, hmap_random_target, analysers = pickle.load(f)

In [ ]:
attr_names = sorted(set([n for (n, _) in hmap_original.keys()]))
attr_names

In [ ]:
def get_sorting(n):
    attr_names_sorting = {
     'DTD': -1,
     'GuidedBP': -1,
     'SmoothGrad': -1,
     '$\\alpha=1, \\beta=0$-LRP': 1,
     '$\\alpha=2, \\beta=1$-LRP': 2,
     '$\\alpha=100, \\beta=99$-LRP': 3,
    }
    if n in attr_names_sorting:
        return attr_names_sorting[n]
    elif "epsilon" in n:
        return 0
    elif "cmp-LRP" in n:
        return 10
    else:
        return 5
    
attr_names = sorted(attr_names, key=lambda x: (get_sorting(x), x))
attr_names

In [ ]:
ssim = OrderedDict()
l2_random_weights = OrderedDict()

last_idx = len(model.layers)
for (name, img_idx, layer_idx), heatmap in tqdm.tqdm_notebook(hmap_random_weights.items()):
    original_heatmap = hmap_original[(name, img_idx)]
    postprocess = hmap_postprocesisng[name]
    original_heatmap = postprocess(original_heatmap)
    heatmap = postprocess(heatmap)
    ssim[(name, img_idx, layer_idx)] = ssim_flipped(heatmap, original_heatmap)
    l2_random_weights[(name, img_idx, layer_idx)] = l2_flipped(heatmap, original_heatmap)

In [ ]:
ssim_random_target = OrderedDict()
for (name, img_idx), (_, hmap_random) in tqdm.tqdm_notebook(hmap_random_target.items()):
    if name not in ssim_random_target:
        ssim_random_target[name] = []
        
    postprocess = hmap_postprocesisng[name]
    
    hmap = hmap_original[name, img_idx]
    original_heatmap = postprocess(original_heatmap)
    hmap = postprocess(hmap)
    hmap_random = postprocess(hmap_random)
    ssim_random_target[name].append(
        ssim_flipped(hmap, hmap_random))

In [ ]:
with sns.axes_style('whitegrid', {"axes.grid": True, 'font.family': 'serif'}):
    fig, ax = plt.subplots(1, 1, figsize=(4, 3.0), squeeze=True)
    mean_ssim = [np.mean(s) for s in ssim_random_target.values()]
    
    names = ssim_random_target.keys()
    bars = ax.bar(names, mean_ssim, 
           color=[linestyles[name]['color'] for name in attr_names])
    
    xlabels = ssim_random_target.keys()
    ax.set_ylabel('SSIM')
    ax.set_xticks(np.arange(len(xlabels)))
    ax.set_xticklabels(xlabels, rotation=90)
    
    
    fig.savefig('check-random-logit.pdf',  bbox_inches='tight', pad_inches=0)
    display(IFrame('check-random-logit.pdf', 800, 500))

In [ ]:
ssim_random_target.keys()

In [ ]:
with sns.axes_style('ticks', {"axes.grid": True, 'font.family': 'serif'}):
    fig, ax = plt.subplots(1, 1, figsize=(4, 3.0), squeeze=True)
    
    
    xlabels = attr_names
    bars = ax.boxplot([ssim_random_target[n] for n in attr_names]) 
    ax.set_ylabel('SSIM')
    #ax.set_xticks(np.arange(len(xlabels)))
    ax.set_xticklabels(xlabels, rotation=90)
    ax.set_ylim(-0.05, 1.05)
    
    fig.savefig('check-random-logit-boxplot.pdf',  bbox_inches='tight', pad_inches=0)
    display(IFrame('check-random-logit-boxplot.pdf', 800, 500))

In [ ]:
ssim_reduce = 'median'
confidence_intervals = True
confidence_percentile = 99.5
with sns.axes_style("ticks", {"axes.grid": True, 'font.family': 'serif'}):
    # metrics = [('SSIM', ssim), ('MSE', l2_random_weights)]
    metrics = [('SSIM', ssim)]
    fig, axes = plt.subplots(1, len(metrics), figsize=(4.5 * len(metrics), 3.5), squeeze=False)
    axes = axes[0]
    for ax, (ylabel, metric) in zip(axes, metrics): 
        for name in attr_names:
            metric_per_layer = []
            
            lower_conf = []
            upper_conf = []
            for (_, layer_idx) in selected_layers[::-1]:
                metric_per_layer.append(
                    [metric[(name, img_idx, layer_idx)] for img_idx in selected_img_idxs]
                )
                if confidence_intervals:
                    vals = np.array(metric_per_layer[-1])
                    ridx = np.random.choice(len(vals), (10000, len(vals)), replace=True)
                    resample = vals[ridx]
                    stats = np.median(resample, 1)
                    lower_conf.append(np.percentile(stats, 100 - confidence_percentile))
                    upper_conf.append(np.percentile(stats, confidence_percentile))
                    
            metric_per_layer = np.array(metric_per_layer)
                
            if ssim_reduce == 'mean':
                ssims_reduced = metric_per_layer.mean(1)
            elif ssim_reduce == 'median':
                ssims_reduced = np.median(metric_per_layer, 1)
            
            ticks = np.arange(len(ssims_reduced))
            ax.plot(ticks, ssims_reduced, label=name, **linestyles[name])
            ax.fill_between(ticks, lower_conf, upper_conf, 
                            color=linestyles[name]['color'],
                            alpha=0.25
                           )
            #ax.plot(ticks, lower_conf, color=linestyles[name]['color'])
            #ax.plot(ticks, upper_conf, color=linestyles[name]['color'])
            
        xlabels = [layer_name for layer_name, _ in selected_layers[::-1]] 
        ax.set_ylim([0, 1.05])
        ax.set_xticks(np.arange(len(xlabels)))
        ax.set_xticklabels(xlabels, rotation=90)
        ax.set_ylabel(ylabel)
    axes[-1].legend(bbox_to_anchor=(1.0, 1.00))
    plt.savefig('check-random-weights.pdf',  bbox_inches='tight', pad_inches=0)
    plt.show()
    display(IFrame('check-random-weights.pdf', 800, 500))

In [ ]:
def plot_heatmap_grid(heatmaps, cols, row_labels=[], column_labels=[], fig_path=None):
    mpl.rcParams['font.family'] = 'serif'
    rows = len(heatmaps) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(cols, rows), squeeze=False)
    fontsize = 12
    plt.subplots_adjust(wspace=0.05, hspace=0.05, top=1, bottom=0, left=0, right=1)
    
    for label, ax in zip(row_labels, axes[:, 0]):
        ax.set_ylabel(label, fontsize=fontsize - 1, labelpad=55, rotation=0)
        
    for label, ax in zip(column_labels, axes[0, :]):
        ax.set_title(label, fontsize=fontsize - 1)
        
        
    for ax, heatmap in zip(axes.flatten(), heatmaps):
        ax.imshow(heatmap, cmap='seismic', vmin=-1, vmax=1)
        ax.set_xticks([])
        ax.set_yticks([])

    #plt.tight_layout()
    if fig_path is not None:
        plt.savefig(fig_path, bbox_inches='tight', pad_inches=0, dpi=150)

In [ ]:
attr_names

In [ ]:
def normalize_neg(x):
    vmax = np.percentile(x, 99)
    return np.clip(x / vmax, -1, 1)

hmap_plot = []
for attr_name in attr_names:
    postp = hmap_postprocesisng[attr_name]
    if attr_name ==  '$\\alpha=1, \\beta=0$-LRP':
        postp_bar = hmap_postprocesisng[attr_name]
        postp = lambda x: flip_hmap(normalize(postp_bar(x)))
        
    for img_idx in selected_img_idxs[:1]:
        hmap_plot.append(norm_image(val_images[0][0][0]))
        hmap_plot.append(normalize_neg(postp(hmap_original[attr_name, img_idx])))
        for (_, layer_idx) in selected_layers[::-1]:
            hmap_plot.append(normalize_neg(postp(hmap_random_weights[attr_name, img_idx, layer_idx])))

plot_heatmap_grid(
    hmap_plot, 2+len(selected_layers), row_labels=attr_names, 
    column_labels=['input', 'original'] + [n for (n, _) in selected_layers][::-1],
    fig_path='heatmap_grid.pdf'
)

In [ ]:
display(IFrame('heatmap_grid.pdf', width=1000, height=600))

In [ ]:
raise Exception("stop ipynb execution")

In [ ]:
list(ssim.keys())[0]

In [ ]:
method = 'DTD'
layer_idx = selected_layers[4][1]
ssim_per_layer = []
for img_idx in selected_img_idxs:
    ssim_per_layer.append(ssim[method, img_idx, layer_idx])
    
ssim_per_layer = np.array(ssim_per_layer)

In [ ]:
means = []
medians = []

ridx = np.random.choice(n_selected_imgs, (10000, n_selected_imgs), replace=True)

means = ssim_per_layer[ridx].mean(1)
median = np.median(ssim_per_layer[ridx], 1)
print(np.percentile(median, 99))
print(np.percentile(median, 1))

## Relevance w.r.t. Parameters


In [ ]:
def prepare_idx(neuron_selection, batch_size=1):
    neuron_selection = np.asarray(neuron_selection).flatten()
    if neuron_selection.size == 1:
        neuron_selection = np.repeat(neuron_selection, batch_size)

    # Add first axis indices for gather_nd
    neuron_selection = np.hstack(
        (np.arange(len(neuron_selection)).reshape((-1, 1)),
         neuron_selection.reshape((-1, 1)))
    )
    return neuron_selection

In [ ]:
rel_wrt_kernels = OrderedDict()
for name, innv_name, innv_kwargs in analysers:
    print(name)
    analyzer = create_analyzer(
        innv_name, model_wo_softmax, neuron_selection_mode="index", **innv_kwargs)
    analyzer.create_analyzer_model()
    try:
        analyzer_model = analyzer._analyzer_model
    except:
        continue
    input_image, input_idx = analyzer_model.inputs

    rel_mean = tf.reduce_sum(analyzer_model.outputs[0])
    rel_grad_w = tf.gradients(
        rel_mean,
        analyzer_model.weights,
    )
    sess = keras.backend.get_session()
    outs = sess.run(analyzer._analyzer_model.outputs + rel_grad_w, 
                    {input_image: ex_image, input_idx: prepare_idx(ex_target)})
    
    output = outs[0]
    rel_grad_w_np = outs[1:]
    print(len(rel_grad_w_np))
    for i, w in enumerate(analyzer_model.weights):
        if "kernel" in w.name:
            #print(i, w.name)
            rel_wrt_kernels[name, ex_image_idx, i, w.name.rstrip("/kernel:0")] = rel_grad_w_np[i]

    print(output.min(), output.max())
    plt.imshow(normalize(output[0]))

In [ ]:
for (name, _, _) in analysers:
    grads = []
    percentile = 99
    lower_percentile = []
    upper_percentile = []
    layer_names = []
    for (gname, _, idx, layer_name), grad in rel_wrt_kernels.items():
        if name == gname:
            grads.append(np.abs(grad).mean())
            lower_percentile.append(np.percentile(np.abs(grad), 100-percentile))
            upper_percentile.append(np.percentile(np.abs(grad), percentile))
            layer_names.append(layer_name)
            
    plt.title(name)
    plt.semilogy(grads)
    plt.semilogy(lower_percentile)
    plt.semilogy(upper_percentile)
    plt.ylim(1e2, 1e-16)
    plt.xticks(ticks=np.arange(len(rel_wrt_kernels)), labels=layer_names, rotation=90)
    plt.show()


In [ ]:
for (name, _, _) in analysers:
    grads = []
    percentile = 99
    lower_percentile = []
    upper_percentile = []
    layer_names = []
    layer_idxs = []
    
    n_layers = 0
    i = 0
    for ((gname, _, idx, layer_name), grad) in rel_wrt_kernels.items():
        if name == gname:
            grads.append(np.abs(grad).flatten())
            layer_idxs.append(i*np.ones_like(grads[-1]))
            layer_names.append(layer_name)
            i += 1
          
    if len(grads) == 0:
        continue
     
    layer_idxs = np.concatenate(layer_idxs)
    grads = np.concatenate(grads)
    print('.')
    
    idx = np.random.choice(len(grads), size=100000)
    print('.')
    
    plt.title(name)
    plt.scatter(layer_idxs[idx], grads[idx], marker='_', alpha=0.1)
    plt.yscale('log')
    plt.ylim([1000, 1e-8])
    plt.xticks(ticks=np.arange(len(layer_names)), labels=layer_names, rotation=90)
    plt.show()

In [ ]:
for (name, _, _) in analysers:
    grads = []
    layer_names = []
    for (gname, _, idx, layer_name), grad in rel_wrt_kernels.items():
        if name == gname:
            grads.append((np.abs(grad) == 0).sum())
            layer_names.append(layer_name)
            
    plt.title(name)
    plt.scatter(np.arange(len(grads)), grads)
    plt.yscale('log')
    plt.ylim(1e9, 1)
    plt.xticks(ticks=np.arange(len(grads)), labels=layer_names, rotation=90)
    plt.show()
